In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [3]:
print(train.shape,test.shape)

(42000, 785) (28000, 784)


In [4]:
#importing required packages
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy




In [5]:
train = tf.convert_to_tensor(train)
test = tf.convert_to_tensor(test)
train = tf.random.shuffle(train, seed=123, name=None)
XY_train, XY_cv = tf.split(train, num_or_size_splits=[35000,7000], axis=0, num=None, name='split')


2022-12-13 08:33:05.517666: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 08:33:05.518626: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 08:33:05.659485: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 08:33:05.660354: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 08:33:05.661163: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [6]:


rotate=tf.keras.layers.RandomRotation(0.05,seed=123)(tf.reshape(XY_train[:,1:],shape=(35000,28,28,1)))
crop=tf.keras.layers.RandomCrop(22, 22, seed=123)(tf.reshape(XY_train[:,1:],shape=(35000,28,28,1)))
crop=tf.image.resize_with_pad(crop,28,28)
crop_rotate = tf.keras.layers.RandomCrop(22, 22, seed=123)(rotate)
crop_rotate = tf.image.resize_with_pad(crop_rotate,28,28)
resize = tf.keras.layers.Resizing(20,20)(tf.reshape(XY_train[:,1:],shape=(35000,28,28,1)))
resize = tf.image.resize_with_pad(resize,28,28)


In [7]:
y_train = tf.concat([XY_train[:,0],XY_train[:,0],XY_train[:,0],XY_train[:,0],XY_train[:,0]],axis=0)
X_train = tf.concat([tf.reshape(XY_train[:,1:],shape=(35000,28,28,1)),tf.cast(crop,'int64'),tf.cast(rotate,'int64'),tf.cast(resize,'int64'),tf.cast(crop_rotate,'int64')],axis=0)

In [8]:
train = tf.concat([tf.reshape(X_train,shape=(175000,-1)),tf.reshape(y_train,shape=(175000,1))],axis=-1)
train = tf.random.shuffle(train,seed=123)
X_train, y_train =  tf.reshape(train[:,:-1],shape=(175000,28,28,1)), tf.reshape(train[:,-1],shape=(175000))

2022-12-13 08:33:11.884485: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1099000000 exceeds 10% of free system memory.
2022-12-13 08:33:12.765598: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1099000000 exceeds 10% of free system memory.


In [9]:

X_train, y_train = X_train,tf.one_hot(y_train,depth=10)
X_cv, y_cv = tf.reshape(XY_cv[:,1:],shape=(-1,28,28,1)),tf.one_hot(XY_cv[:,0],depth=10)
X_test = tf.reshape(test[:,:],shape=(-1,28,28,1))

In [10]:
pad_0 = tf.keras.layers.ZeroPadding2D(padding=((2,2),(2,2)))
X_train = pad_0(X_train)
X_cv = pad_0(X_cv)
X_test = pad_0(X_test)



2022-12-13 08:33:13.932938: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1097600000 exceeds 10% of free system memory.
2022-12-13 08:33:14.815070: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1433600000 exceeds 10% of free system memory.


In [11]:
def resnet50():
    base_model = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet',input_shape=(32,32,3))
    for layer in base_model.layers[:41]:
        layer.trainable=False
    
    inputs = tf.keras.Input(shape=(32,32,1))
    X = tf.keras.layers.Conv2D(filters=3,kernel_size=(1,1),strides=(1,1),padding='valid')(inputs)
    X = tf.keras.applications.resnet50.preprocess_input(X)
    X = base_model(X,training=False)
    X = tf.keras.layers.GlobalAveragePooling2D()(X)
    X = tf.keras.layers.Dropout(0.2)(X)
    outputs = tf.keras.layers.Dense(units=10, activation='sigmoid')(X)
    model = tf.keras.Model(inputs=inputs,outputs=outputs)
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer= tf.keras.optimizers.Adam(0.00001), metrics=['Accuracy'])
    model.fit(X_train,y_train, epochs=25, batch_size=32)
    return model



In [12]:
model = resnet50()
print('train',(sum(tf.keras.metrics.categorical_accuracy(y_train,model.predict(X_train)))/len(X_train)).numpy(),'cv',(sum(tf.keras.metrics.categorical_accuracy(y_cv,model.predict(X_cv)))/len(X_cv)).numpy())


94781440/94765736 [==============================] - 0s 0us/step


2022-12-13 08:33:18.826738: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25


2022-12-13 08:33:24.628488: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


5469/5469 [==============================] - 236s 41ms/step - loss: 0.1116 - Accuracy: 0.9654
Epoch 2/25
5469/5469 [==============================] - 220s 40ms/step - loss: 0.0270 - Accuracy: 0.9915
Epoch 3/25
5469/5469 [==============================] - 223s 41ms/step - loss: 0.0161 - Accuracy: 0.9949
Epoch 4/25
5469/5469 [==============================] - 218s 40ms/step - loss: 0.0115 - Accuracy: 0.9962
Epoch 5/25
5469/5469 [==============================] - 216s 40ms/step - loss: 0.0090 - Accuracy: 0.9972
Epoch 6/25
5469/5469 [==============================] - 216s 40ms/step - loss: 0.0069 - Accuracy: 0.9979
Epoch 7/25
5469/5469 [==============================] - 216s 40ms/step - loss: 0.0055 - Accuracy: 0.9983
Epoch 8/25
5469/5469 [==============================] - 217s 40ms/step - loss: 0.0050 - Accuracy: 0.9984
Epoch 9/25
5469/5469 [==============================] - 218s 40ms/step - loss: 0.0044 - Accuracy: 0.9987
Epoch 10/25
5469/5469 [==============================] - 218s 40ms

In [13]:


dataset = pd.DataFrame({'ImageId':np.arange(1,28001),'Label':np.argmax(model.predict(X_test),axis=1)})
dataset.to_csv('submission.csv',index=False)

